In [1]:
import torch
from torch import nn
from torch.nn import MSELoss
from torch.optim import Adam
from LSTM_model.data_loader import StockData
from LSTM_model.model import StockModel
import pandas as pd
import numpy as np

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else: 
    device = "cpu"

In [3]:
data = pd.read_csv("stock_data.csv")
company_name = "MSFT"
data = data[data["Stock"] == company_name]

In [4]:
from sklearn.preprocessing import MinMaxScaler

price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price = scaler.fit_transform(price['Close'].values.reshape(-1,1))

In [5]:
def split_data(stock, lookback):
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(stock) - lookback): 
        data.append(stock[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [6]:
lookback = 20
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [7]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)

y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [8]:
input_dim = 1
hidden_dim = 128
num_layers = 2
output_dim = 1
num_epochs = 500

In [9]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        out, (hn) = self.gru(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

In [10]:
gru_model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).to(device)
criterion = torch.nn.MSELoss(reduction='mean').to(device)
optimiser = torch.optim.Adam(gru_model.parameters(), lr=0.01)

In [11]:
import math, time
from sklearn.metrics import mean_squared_error

def validate(model, x_test, y_test):
    # make predictions
    y_test_pred = model(x_test.cuda()).cpu()

    # invert predictions
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test.detach().numpy())

    # calculate root mean squared error
    testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
    
    return testScore

In [12]:
import time

hist = np.zeros(num_epochs)
start_time = time.time()
gru = []

save_losses = {}
save_losses["min_test_loss"] = 10e10
for epoch in range(num_epochs):
    y_train_pred = gru_model(x_train.cuda())

    loss = criterion(y_train_pred.cuda(), y_train_gru.cuda())
    test_loss = validate(gru_model, x_test, y_test_gru)
    hist[epoch] = loss.item()
    print(f"Epoch {epoch} - MSE: {loss.item()} - Test MSE: {test_loss}")
    if save_losses["min_test_loss"] > test_loss:
        save_losses["min_test_loss"] = test_loss
        save_losses["train_loss"] = loss.item()
        save_losses["epoch"] = epoch
        
        # torch.save(gru_model.state_dict(), f"checkpoints/{company_name}/{epoch}_gru128_{test_loss}.pth")
        torch.save(gru_model.state_dict(), f"checkpoint/{epoch}_gru128_{test_loss}.pth")

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

training_time = time.time()-start_time    
print("Training time: {}".format(training_time))

Epoch 0 - MSE: 0.501528263092041 - Test MSE: 14.998620033365517
Epoch 1 - MSE: 0.11834418028593063 - Test MSE: 36.109648825227445
Epoch 2 - MSE: 0.2036104053258896 - Test MSE: 18.21958583792367
Epoch 3 - MSE: 0.1440645158290863 - Test MSE: 18.10168978564441
Epoch 4 - MSE: 0.0469244047999382 - Test MSE: 20.70298201601568
Epoch 5 - MSE: 0.06691712141036987 - Test MSE: 25.513543814389056
Epoch 6 - MSE: 0.05249348282814026 - Test MSE: 23.686801708869698
Epoch 7 - MSE: 0.026221532374620438 - Test MSE: 19.01472224002324
Epoch 8 - MSE: 0.04857770726084709 - Test MSE: 15.99517081815487
Epoch 9 - MSE: 0.04202905297279358 - Test MSE: 14.302314432290922
Epoch 10 - MSE: 0.012662939727306366 - Test MSE: 13.025854956158048
Epoch 11 - MSE: 0.023357948288321495 - Test MSE: 11.618581661258656
Epoch 12 - MSE: 0.022673428058624268 - Test MSE: 8.819779223665364
Epoch 13 - MSE: 0.0019014566205441952 - Test MSE: 5.57118661236324
Epoch 14 - MSE: 0.018636338412761688 - Test MSE: 4.219942277631068
Epoch 15 - M

In [13]:
print(save_losses)

{'min_test_loss': 0.9205439705549139, 'train_loss': 8.49300849949941e-05, 'epoch': 423}
